In [1]:
folder = '/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/xgBoost-Results-ArpitNewFeatures_V1_2011/'
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
for files in os.listdir(folder):
    print(files)

MSW_2011AnujMethod2011__top_score_specs@11.pkl
MSW_2011AnujMethod2011__top_score_specs@10.pkl
MSW_2011AnujMethod2011__top_score_specs@12.pkl
MSW_2011AnujMethod2011__top_score_specs@13.pkl
ASSET_2011AnujMethod2011__top_score_specs@9.pkl
BF_2011AnujMethod2011__top_score_specs@8.pkl
MSW_2011AnujMethod2011__top_score_specs@14.pkl
MSW_2011AnujMethod2011__top_score_specs@15.pkl
BF_2011AnujMethod2011__top_score_specs@9.pkl
ASSET_2011AnujMethod2011__top_score_specs@8.pkl
CHH_2011AnujMethod2011__top_score_specs@5.pkl
ASSET_2011AnujMethod2011__top_score_specs@11.pkl
FC_2011AnujMethod2011__top_score_specs@8.pkl
MSL_2011AnujMethod2011__top_score_specs@8.pkl
FC_2011AnujMethod2011__top_score_specs@9.pkl
MSL_2011AnujMethod2011__top_score_specs@9.pkl
ASSET_2011AnujMethod2011__top_score_specs@10.pkl
CHH_2011AnujMethod2011__top_score_specs@6.pkl
ASSET_2011AnujMethod2011__top_score_specs@12.pkl
ASSET_2011AnujMethod2011__top_score_specs@13.pkl
CHH_2011AnujMethod2011__top_score_specs@7.pkl
MSW_2011AnujMeth

In [5]:
for files in os.listdir(folder):
    filepath = os.path.join(folder, files)
    if 'CHH' in files:
        print(files)
        tss = pickle.load(open(filepath, 'rb'))
        print(tss['xgBoost']['val_scores'])
        print(tss['xgBoost']['train_scores'])
        print('-------------------------')

CHH_2011AnujMethod2011__top_score_specs@5.pkl
[0.7106460146944565, 0.7078114819839667, 0.7066314462698148, 0.6971259584551296, 0.6740703626315139]
[0.9974555600375605, 0.9974553503315372, 0.9974553401514987, 0.9974556680037827, 0.9974553468837772]
-------------------------
CHH_2011AnujMethod2011__top_score_specs@6.pkl
[0.7175904657770893, 0.6979898470739452, 0.6937205583653423, 0.6866120298006647, 0.6763944484244744]
[0.9974553503315372, 0.9974552349419417, 0.9974555600375605, 0.9974556680037827, 0.9974553468837772]
-------------------------
CHH_2011AnujMethod2011__top_score_specs@7.pkl
[0.7192441485119603, 0.7022775853662395, 0.698448859601601, 0.6875750789879038, 0.6677756260556315]
[0.9974553503315372, 0.9974553401514987, 0.9974556680037827, 0.9974555600375605, 0.9974553468837772]
-------------------------
CHH_2011AnujMethod2011__top_score_specs@9.pkl
[0.7136290578986809, 0.704996138361293, 0.7044609784778533, 0.6977232803834255, 0.6775930761806772]
[0.9974553503315372, 0.9974555600

In [30]:
tss = pickle.load(open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/CHH_2011AnujMethod2011_SVC_top_score_specs@9.pkl",'rb'))

In [29]:
print(tss['randomForest']['val_scores'])
print(tss['randomForest']['train_scores'])

[0.6699429267160563, 0.6473250340055784, 0.6103361182620473, 0.5929448332344716, 0.5558500000247654]
[0.8962247328381535, 0.9226714760162948, 0.7873919140475604, 0.8001958173021624, 0.6988138357322536]


In [31]:
print(tss['svc']['val_scores'])
print(tss['svc']['train_scores'])

[0.6478170910635415, 0.6260063203503232, 0.6249848812607371, 0.620035047335605, 0.5857395985853994]
[0.8887401690167194, 0.8872017283805216, 0.8904608071701314, 0.8924468501285162, 0.889171896771774]


In [3]:
def apply_smote(data, feature_cols, target):
    '''
    Input:
    data - the original dataframe
    feature_cols - the feature columns (list of columns)
    target - the target column (string value)
    '''
    sm = SMOTE(random_state=42)
    features, targets = sm.fit_resample(data[feature_cols],data[target])
    feature_df = pd.DataFrame(features, columns=feature_cols)
    target_df = pd.DataFrame(targets, columns=[target])
    output = pd.concat([feature_df, target_df], axis=1)
    # Shuffling dataset
    output = output.sample(frac=1).reset_index(drop=True)
    return output

In [4]:
def return_filepath(year, bincount):
    a = '/Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@5.csv'
    filepath = a.replace('2001', str(year))
    filepath = filepath.replace('@5', '@'+str(bincount))
#     print(filepath)
    return filepath

In [6]:
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
cols = ['census_code','MSW_2001','BF_2001','MSL_2001', 'FC_2001','CHH_2001','ASSET_2001']
ground_truth = ground_truth[cols]

In [8]:
ground_truth['ASSET_2001'].value_counts()

1    469
2     89
3     35
Name: ASSET_2001, dtype: int64

In [6]:
def get_results(indicator, year, folder):
    print('###   Running for ', indicator)
    
    dict_ = {}
    for file in os.listdir(folder):
        if year in file:
            if indicator in file:
                bincount = file.split('@')[1].split('.')[0]

                filepath = return_filepath(year, bincount)
                print('     ### Extracting features from ',filepath)
                df = pd.read_csv(filepath)
                # For 2001
                ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
                cols = ['census_code','MSW_2001','BF_2001','MSL_2001', 'FC_2001','CHH_2001','ASSET_2001']
                ground_truth = ground_truth[cols]
                data = ground_truth.merge(df, how='left', on='census_code')
                data.dropna(inplace=True)
                
                feature_cols = data.columns[7:].tolist()
                y_col = indicator + '_' + year
                output = apply_smote(data, feature_cols, y_col)
                X = output[feature_cols].values
                y = output[y_col].values
                
                print('           ',feature_cols,y_col)
                print('------->>>',y_col)
                print('-------------')
                
                tss = pickle.load( open(os.path.join(folder, file), 'rb'))
                print('Loading model from - ',file)
                print('val_f1_score ',tss['xgBoost']['val_scores'])
                print('train_f1_score ',tss['xgBoost']['train_scores'])

                n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
                max_depth = tss['xgBoost']['specs'][0]['max_depth']
                learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
                objective = tss['xgBoost']['specs'][0]['objective']
                booster = tss['xgBoost']['specs'][0]['booster']
                gamma = tss['xgBoost']['specs'][0]['gamma']
                min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
                max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
                subsample = tss['xgBoost']['specs'][0]['subsample']
                colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
                colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
                colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
                reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
                reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
                scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
                base_score = tss['xgBoost']['specs'][0]['base_score']

                n_splits = tss['xgBoost']['specs'][0]['kFold_splits']
                
                print('                 ')
                print('Re-Training again for calculating f1 and train scores')

                xgbc = XGBClassifier(n_estimators=n_estimators, 
                         max_depth=max_depth, learning_rate=learning_rate, 
                         objective=objective, booster=booster,n_jobs=-1, 
                         gamma=gamma, min_child_weight=min_child_weight, 
                         max_delta_step=max_delta_step, subsample=subsample, 
                         colsample_bytree=colsample_bytree, 
                         colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
                         reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
                         scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

                cv = KFold(n_splits=n_splits, shuffle=True)

                val_f1score = []
                val_accscore = []
                train_f1score = []
                train_accscore = []
                
#                 for train_index, test_index in cv.split(X):
#                     X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
#                     best_xgb = xgbc.fit(X_train, y_train)

#                     predictions = best_xgb.predict(X_test)
#                     predictions_train = best_xgb.predict(X_train)

#                     f1_weight = f1_score(y_test, predictions, average='weighted')
#                     f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
#                     acc = accuracy_score(y_test, predictions)
#                     acc_train = accuracy_score(y_train, predictions_train)

#                     val_f1score.append(f1_weight)
#                     train_f1score.append(f1_weight_train)
#                     val_accscore.append(f1_weight)
#                     train_accscore.append(f1_weight_train)

#                 val_f1score = np.array(val_f1score).mean()
#                 train_f1score = np.array(train_f1score).mean()
#                 val_accscore = np.array(val_accscore).mean()
#                 train_accscore = np.array(train_accscore).mean()
                
#                 dict_key = str(year) + '@' + str(indicator) + '@' + str(bincount)
#                 dict_[dict_key] = [val_f1score, train_f1score, val_accscore, train_accscore, tss['xgBoost']['val_scores'], tss['xgBoost']['train_scores']]

                
#                 print('~~~~~~~~~~~~')
#                 print('~~~~~~~~~~~~New_Scores : val_f1, train_f1, val_acc, train_acc')
#                 print(val_f1score, train_f1score, val_accscore, train_accscore)
                print('******************************************')
    return dict_


In [7]:
bf_dict = get_results('BF','2001',folder)

###   Running for  BF
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@9.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5_8', 'band_5_9', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_6_7', 'band_6_8', 'band_6_9', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_7_7', 'band_7_8', 'band_7_9', 'band_8_1', 'band_8_2', 'band_8_3', 'band_8_4', 'band_8_5', 'band_8_6', 'band_8_7

            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_1_11', 'band_1_12', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_2_11', 'band_2_12', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_3_11', 'band_3_12', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_4_10', 'band_4_11', 'band_4_12', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5_8', 'band_5_9', 'band_5_10', 'band_5_11', 'band_5_12', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_6_7', 'band_6_8', 'band_6_9', 'band_6_10', 'band_6_11', 'band_6_12', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_7_7', 'band_7_8', 'band_7_9

In [8]:
FC_dict = get_results('FC','2001',folder)

###   Running for  FC
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@7.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_6_7', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_7_7', 'band_8_1', 'band_8_2', 'band_8_3', 'band_8_4', 'band_8_5', 'band_8_6', 'band_8_7', 'band_9_1', 'band_9_2', 'band_9_3', 'band_9_4', 'band_9_5', 'band_9_6', 'band_9_7', 'band_10_1', 'band_10_2', 'band_10_3', 'band_10_4', 'band_10_5', 'band_10_6', 'ba

Loading model from -  FC_2001AnujMethod__top_score_specs@10.pkl
val_f1_score  [0.8827776242124707, 0.8750516554763857, 0.8691786883601071, 0.8627925647048122, 0.8583336442353835]
train_f1_score  [0.9966512698971932, 0.9968899393795091, 0.9968901635315394, 0.9966510054712078, 0.9966515780908315]
                 
Re-Training again for calculating f1 and train scores
******************************************
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@11.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_1_11', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_2_11', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_3_11', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4

In [9]:
MSW_dict = get_results('MSW','2001',folder)

###   Running for  MSW
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@15.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_1_11', 'band_1_12', 'band_1_13', 'band_1_14', 'band_1_15', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_2_11', 'band_2_12', 'band_2_13', 'band_2_14', 'band_2_15', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_3_11', 'band_3_12', 'band_3_13', 'band_3_14', 'band_3_15', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_4_10', 'band_4_11', 'band_4_12', 'band_4_13', 'band_4_14', 'band_4_15', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5

            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_8_1', 'band_8_2', 'band_8_3', 'band_8_4', 'band_8_5', 'band_8_6', 'band_9_1', 'band_9_2', 'band_9_3', 'band_9_4', 'band_9_5', 'band_9_6', 'band_10_1', 'band_10_2', 'band_10_3', 'band_10_4', 'band_10_5', 'band_10_6', 'band_11_1', 'band_11_2', 'band_11_3', 'band_11_4', 'band_11_5', 'band_11_6', 'band_12_1', 'band_12_2', 'band_12_3', 'band_12_4', 'band_12_5', 'band_12_6'] MSW_2001
------->>> MSW_2001
-------------
Loading model from -  MSW_2001AnujMethod__top_score_specs@6.pk

In [10]:
MSW_dict = get_results('MSL','2001',folder)

###   Running for  MSL
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@6.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_8_1', 'band_8_2', 'band_8_3', 'band_8_4', 'band_8_5', 'band_8_6', 'band_9_1', 'band_9_2', 'band_9_3', 'band_9_4', 'band_9_5', 'band_9_6', 'band_10_1', 'band_10_2', 'band_10_3', 'band_10_4', 'band_10_5', 'band_10_6', 'band_11_1', 'band_11_2', 'band_11_3', 'band_11_4', 'band_11_5', 'band_11_6', 'band_12_1', 'band_12_2', 'band_

            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_4_10', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5_8', 'band_5_9', 'band_5_10', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_6_7', 'band_6_8', 'band_6_9', 'band_6_10', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_7_7', 'band_7_8', 'band_7_9', 'band_7_10', 'band_8_1', 'band_8_2', 'band_8_3', 'band_8_4', 'band_8_5', 'band_8_6', 'band_8_7', 'band_8_8', 'band_8_9', 'band_8_10', 'band_9_1', 'band_9

In [11]:
CHH_dict = get_results('CHH','2001',folder)

###   Running for  CHH
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@14.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_1_11', 'band_1_12', 'band_1_13', 'band_1_14', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_2_11', 'band_2_12', 'band_2_13', 'band_2_14', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_3_11', 'band_3_12', 'band_3_13', 'band_3_14', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_4_10', 'band_4_11', 'band_4_12', 'band_4_13', 'band_4_14', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5_8', 'band_5_9', 'band_5_10', 'band_5_11', 'band_5_1

Loading model from -  CHH_2001AnujMethod__top_score_specs@7.pkl
val_f1_score  [0.7584075572701499, 0.74363102256618, 0.7401501145101052, 0.729787535256993, 0.7249940400216829]
train_f1_score  [1.0, 1.0, 1.0, 1.0, 1.0]
                 
Re-Training again for calculating f1 and train scores
******************************************
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@6.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_8_1', 'band_8_2'

In [12]:
ASSET_dict = get_results('ASSET','2001',folder)

###   Running for  ASSET
     ### Extracting features from  /Users/arpitjain/Downloads/SatPRo/2001_L7_data/2001_Anuj_Method/Features_100m_quantile@14.csv
            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_1_11', 'band_1_12', 'band_1_13', 'band_1_14', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_2_11', 'band_2_12', 'band_2_13', 'band_2_14', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_3_11', 'band_3_12', 'band_3_13', 'band_3_14', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_4_10', 'band_4_11', 'band_4_12', 'band_4_13', 'band_4_14', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5_8', 'band_5_9', 'band_5_10', 'band_5_11', 'band_5

            ['band_1_1', 'band_1_2', 'band_1_3', 'band_1_4', 'band_1_5', 'band_1_6', 'band_1_7', 'band_1_8', 'band_1_9', 'band_1_10', 'band_1_11', 'band_2_1', 'band_2_2', 'band_2_3', 'band_2_4', 'band_2_5', 'band_2_6', 'band_2_7', 'band_2_8', 'band_2_9', 'band_2_10', 'band_2_11', 'band_3_1', 'band_3_2', 'band_3_3', 'band_3_4', 'band_3_5', 'band_3_6', 'band_3_7', 'band_3_8', 'band_3_9', 'band_3_10', 'band_3_11', 'band_4_1', 'band_4_2', 'band_4_3', 'band_4_4', 'band_4_5', 'band_4_6', 'band_4_7', 'band_4_8', 'band_4_9', 'band_4_10', 'band_4_11', 'band_5_1', 'band_5_2', 'band_5_3', 'band_5_4', 'band_5_5', 'band_5_6', 'band_5_7', 'band_5_8', 'band_5_9', 'band_5_10', 'band_5_11', 'band_6_1', 'band_6_2', 'band_6_3', 'band_6_4', 'band_6_5', 'band_6_6', 'band_6_7', 'band_6_8', 'band_6_9', 'band_6_10', 'band_6_11', 'band_7_1', 'band_7_2', 'band_7_3', 'band_7_4', 'band_7_5', 'band_7_6', 'band_7_7', 'band_7_8', 'band_7_9', 'band_7_10', 'band_7_11', 'band_8_1', 'band_8_2', 'band_8_3', 'band_8_4', '

In [3]:
for file in os.listdir(folder):
    
    if '2001' in files:
        year = 2001
        if 'MSW' in files:
            indicator ='MSW'
            bincount = file.split('@')[1].split('.')[0]
            print(bincount, files)
            
            filepath = return_filepath(year, bincount)
            df = pd.read_csv(filepath)
            # For 2001
            ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
            cols = ['census_code','MSW_2001','BF_2001','MSL_2001', 'FC_2001','CHH_2001','ASSET_2001']
            ground_truth = ground_truth[cols]
            data = ground_truth.merge(df, how='left', on='census_code')
            data.dropna(inplace=True)
            
            y_col = 'MSW'
            output = apply_smote(data, feature_cols, y_col)
            X = output[feature_cols].values
            y = output[y_col].values
            
            
            
            tss = pickle.load( open(os.path.join(folder, file)), 'rb')
            print(tss['xgBoost']['val_scores'])
            print(tss['xgBoost']['train_scores'])
            
            n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
            max_depth = tss['xgBoost']['specs'][0]['max_depth']
            learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
            objective = tss['xgBoost']['specs'][0]['objective']
            booster = tss['xgBoost']['specs'][0]['booster']
            gamma = tss['xgBoost']['specs'][0]['gamma']
            min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
            max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
            subsample = tss['xgBoost']['specs'][0]['subsample']
            colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
            colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
            colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
            reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
            reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
            scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
            base_score = tss['xgBoost']['specs'][0]['base_score']
            
            n_splits = tss['xgBoost']['specs'][0]['kFold_splits']
            
            xgbc = XGBClassifier(n_estimators=n_estimators, 
                     max_depth=max_depth, learning_rate=learning_rate, 
                     objective=objective, booster=booster,n_jobs=-1, 
                     gamma=gamma, min_child_weight=min_child_weight, 
                     max_delta_step=max_delta_step, subsample=subsample, 
                     colsample_bytree=colsample_bytree, 
                     colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
                     reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
                     scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

            cv = KFold(n_splits=n_splits, shuffle=True)
    
            val_f1score = []
            val_accscore = []
            train_f1score = []
            train_accscore = []
            for train_index, test_index in cv.split(X):
                X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
                best_xgb = xgbc.fit(X_train, y_train)

                predictions = best_xgb.predict(X_test)
                predictions_train = best_xgb.predict(X_train)
                
                f1_weight = f1_score(y_test, predictions, average='weighted')
                f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
                acc = accuracy_score(y_test, predictions)
                acc_train = accuracy_score(y_train, predictions_train)

                val_f1score.append(f1_weight)
                train_f1score.append(f1_weight_train)
                val_accscore.append(f1_weight)
                train_accscore.append(f1_weight_train)
            
            val_f1score = np.array(val_f1score).mean()
            train_f1score = np.array(train_f1score).mean()
            val_accscore = np.array(val_accscore).mean()
            train_accscore = np.array(train_accscore).mean()
                
                
            
        elif 'MSL' in files:
            indicator ='MSL'
        elif 'MSW' in files:
            indicator ='MSW'
        elif 'MSW' in files:
            indicator ='MSW'
        elif 'MSW' in files:
            indicator ='MSW'
        elif 'MSW' in files:
            indicator ='MSW'
        elif 'MSW' in files:
            indicator ='MSW'
    
    elif '2011' in files:
        year =2011

10 MSW_2001AnujMethod__top_score_specs@10.pkl
11 MSW_2001AnujMethod__top_score_specs@11.pkl
12 MSW_2001AnujMethod__top_score_specs@12.pkl
